#### LIVRABLE PROJET PYTHON (Marius ayrault)

#### INPUT
1) Import des libraires

In [16]:
import os
from datetime import datetime 
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go

2) Création des dataframes à partir des fichiers csv

In [17]:
def load_data_from_csv():

    notebook_dir = os.getcwd()                          #Chemin absolu du notebook         
    data_dir = os.path.join(notebook_dir, "../data")    #Construction du chemin absolu vers le dossier 'data'
    csv_files = os.listdir(data_dir)                    #Liste des fichiers dans le dossier 'data'

    dataframes_dict = {}          #Déclaration dictionnaire pour contenir les dataframes
    
    for file in csv_files:           #Boucle pour chaque fichier .csv du dossier 'data' : 
        if file.endswith('.csv'):
            
            file_path = os.path.join(data_dir, file) 
            df = pd.read_csv(file_path)                 #Création du dataframe

            file_name = os.path.splitext(file)[0]
            dataframes_dict['df_' + str(file_name)] = df    #Renommage du dataframe avec préfixe "df_" + 'nom_du_fichier'
            
            df['timestamp'] = pd.to_datetime(df['timestamp'])   #Conversion 'timestamp' en type Datetime    

            print(f"df_{file_name}")      #Affichage du df créé pour vérification
            print(df.head(3))   
            print("\n")

    return dataframes_dict

dataframes_dict = load_data_from_csv()

df_stmp
            timestamp    price  amount
0 2021-02-24 23:59:54  49754.0   0.753
1 2021-02-24 23:59:52  49754.0   0.116
2 2021-02-24 23:59:52  49754.0   0.104


df_lmax
                timestamp    price  amount
0 2021-02-24 23:59:59.691  49767.0    0.01
1 2021-02-24 23:59:42.786  49752.0    0.06
2 2021-02-24 23:59:42.785  49752.0    0.30


df_gmni
                timestamp     price    amount
0 2021-02-24 23:59:53.406  49773.07  0.003435
1 2021-02-24 23:59:53.406  49766.06  0.051690
2 2021-02-24 23:59:47.280  49746.16  0.122833


df_itbi
                timestamp     price  amount
0 2021-02-24 23:59:48.157  49753.50  0.0001
1 2021-02-24 23:59:45.463  49753.75  0.0004
2 2021-02-24 23:59:38.887  49734.50  0.0001


df_okcn
                timestamp     price  amount
0 2021-02-24 23:59:57.847  49724.93    0.02
1 2021-02-24 23:59:49.743  49730.33    0.02
2 2021-02-24 23:59:35.623  49706.87    0.02


df_bfnx
                timestamp    price  amount
0 2021-02-24 23:59:58.181  49716.0 

#### MANIPULATION DES DONNEES

1) Création d'un dataframe 'df_all' et intégration dans le dictionnaire

In [18]:
def global_df_creation(dataframes_dict): 
    
    dfs_to_concat = list(dataframes_dict.values())     #Identification des dataframes à concaténer
    print(f"Concaténation des dataframes :\n{list(dataframes_dict)}\n")
    
    df_all = pd.concat(dfs_to_concat, ignore_index=True)     #Concaténation verticale -> pas de perte de données avec la fonction concat, redéfinition des index pour avoir une clé unique par transaction
    print("'df_all' :")     
    display(df_all)     #Vérification : '2021-02-24 23:59:52.000' *2 dans le display = transactions conservées      

    all_dataframes_dict = dataframes_dict.copy()
    all_dataframes_dict['df_all'] = df_all
    print(f"Intégration dans le dictionnaire 'all_dataframes_dict' :\n{list(all_dataframes_dict)}\n")       #Vérification

    return all_dataframes_dict
    
all_dataframes_dict = global_df_creation(dataframes_dict) 

Concaténation des dataframes :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn']

'df_all' :


,timestamp,price,amount
0,2021-02-24 23:59:54.000,49754.0,0.753000
1,2021-02-24 23:59:52.000,49754.0,0.116000
2,2021-02-24 23:59:52.000,49754.0,0.104000
3,2021-02-24 23:59:49.000,49754.0,0.016000
4,2021-02-24 23:59:45.000,49754.0,0.011000
...,...,...,...
1136788,2021-02-24 00:00:11.182,48899.8,0.023270
1136789,2021-02-24 00:00:10.373,48899.9,0.200000
1136790,2021-02-24 00:00:07.818,48899.9,0.018278
1136791,2021-02-24 00:00:02.351,48899.9,0.002045


Intégration dans le dictionnaire 'all_dataframes_dict' :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn', 'df_all']



#### CALCUL

1) Définition des paramètres de calcul

In [19]:
def get_frequency():    #Définition de la période d'aggrégation
    
    while True:
        try:
            frequency_input = input("Veuillez saisir une valeur numérique pour la fréquence (en minutes) : ")
            frequency = int(frequency_input)
            frequency_str = str(frequency) + 'min'
            return frequency_str
        
        except ValueError:
            print("Erreur : Veuillez saisir une valeur numérique valide.")

def get_vwmp_type():    #Définition du calcul de VWMP (lower or upper)
   
    while True:
        vwmp_type = input("Veuillez saisir 'lower' ou 'upper' pour le calcul du VWMP : ").lower()
        if vwmp_type in ['lower', 'upper']:
            return vwmp_type
        
        else:
            print("Erreur : Veuillez saisir 'lower' ou 'upper'.")       

2) Agrégation des données et calcul selon paramètres sélectionnés

In [20]:
def process_data(all_dataframes_dict, frequency, vwmp_type): #PROCESS DE CALCUL                                                    
    
    print(f"Fréquence sélectionnée : {frequency}")
    print(f"Calcul VWMP sélectionné : {vwmp_type}")
    print("\n...calcul en cours")
    
    aggregated_dict = {}        #Déclation des dictionnaires (plusieurs dictionnaires pour vérifications intermédiaires)
    calculated_dict = {}        
    cleaned_dict = {}           
    compilated_dict = {}                    

    for key, df in all_dataframes_dict.items():             

        exchange_name = (str(key)).split("df_")[1]                                         
        
        aggregated_df = aggregate_data(df, frequency)     #(1)Aggrégation des données initiales
        aggregated_dict[key] = aggregated_df 
        
        calculated_df = calculate_metrics(df, frequency, vwmp_type, aggregated_df, exchange_name)     #(2)Calcul des métriques
        calculated_dict[key] = calculated_df 
                
        cleaned_df = clean_data(calculated_df, exchange_name)     #(3)Suppression des lignes vides
        cleaned_dict[key] = cleaned_df

    compilated_dict = compilate_data(cleaned_dict, vwmp_type)      #(4)Compilation des résultats dans un 'df_synthese'
    
    clear_output()
    print("Calcul terminé !\n")
    print(f"Fréquence d'aggrégation : {frequency}")
    print(f"Calcul du VWMP : {vwmp_type}")
    
    return aggregated_dict, calculated_dict, cleaned_dict, compilated_dict

#--------------------------------------------------------------------------------------------------------------------------------

def aggregate_data(df, frequency):     #(1) AGGRÉGATION DES DONNÉES INITIALES (Price, Amount)                  
    
    df['weighted_volume'] = (df['price'] * df['amount'])        #Calcul du volume de chaque transaction

    aggregated_df = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).agg({    #Fonctions d'agrégation pour chaque colonnes
        'price': ['sum', 'first', 'max', 'min', 'last'],   
        'amount': 'sum',
        'weighted_volume': 'sum'
    })

    aggregated_df.columns = ['price', 'price_open', 'price_high', 'price_low', 'price_close','amount','weighted_volume']     #Renommage des colonnes
    
    return aggregated_df

def calculate_metrics(df, frequency, vwmp_type, aggregated_df, exchange_name):   #(2) CALCUL DES MÉTRIQUES (VWAP, VWMP, Ecart_type)                      
    
    calculated_df = aggregated_df.copy()     
    calculated_df[f'{exchange_name}_vwap'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_vwap)     #Calcul du Volume Weighted Average Price [VWAP] 
    calculated_df['sigma'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_ecart_type)     #Calcul de l'écart_type 
    calculated_df[f'{exchange_name}_vwmp ({vwmp_type})'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(lambda group: pd.Series(calculate_vwmp(group, vwmp_type)))     #Calcul du Volume Weighted Median Price [VWMP (lower or upper)] 

    return calculated_df 

def calculate_vwap(group):     #(2.1) CALCUL DU VOLUME WEIGHTED AVERAGE PRICE [VWAP]                   
    
    sum_price_amount = group['weighted_volume'].sum()                                           
    sum_amount = group['amount'].sum()

    if sum_amount != 0:
        return (sum_price_amount / sum_amount)
    else : 
        return 0    

def calculate_ecart_type(group):     #(2.2) CALCUL DE L'ÉCART TYPE                                          
    
    ecart_type = np.nanstd(group['price'])      #'np.nanstd' pour exclure les valeurs nulles dans le calcul

    if ecart_type != np.nan :
        return ecart_type
    else : 
        return 0        #Nota : écart type à zéro dans le cas d'un échantillon de taille 1 (une seule valeur de 'price' sur la période)

def calculate_vwmp(group, vwmp_type):     #(2.3) CALCUL DU VOLUME WEIGHTED MEDIAN PRICE [VWMP (lower or upper)]   

    series_sorted = group.sort_values('amount')                         #Tri du volume par ordre croissant
    series_sorted['cumul_amount'] = series_sorted['amount'].cumsum()      #Calcul du volume cumulé
    total_volume_median = (series_sorted['cumul_amount'].max() / 2)      #Calcul de la médiane
    
    if vwmp_type == 'lower':    #LOWER
       
        lower_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (<=) du volume cumulé 
       
        if not lower_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
           
            max_cumul_amount_index = lower_cumulative_volume['cumul_amount'].idxmax()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_lower = group.loc[max_cumul_amount_index, 'price']     #Déduction du prix médian bas pondéré par le volume
            return round(vwmp_lower, 2)  
        
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) --> (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat souhaité

    #-- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - #symétrie

    if vwmp_type == 'upper':    #UPPER
    
        upper_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (>=) du volume cumulé 
    
        if not upper_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
       
            min_cumul_amount_index = upper_cumulative_volume['cumul_amount'].idxmin()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_upper = group.loc[min_cumul_amount_index, 'price']     #Déduction du prix médian haut pondéré par le volume
            return round(vwmp_upper, 2)  
    
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) --> (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat souhaité 
    
    else:
        raise ValueError("Mode non valide. Veuillez spécifier 'lower' ou 'upper'.")

def clean_data(calculated_df, exchange_name):     #(3) SUPPRESSION DES LIGNES VIDES (intervalle de temps sans transaction)   
    
    cleaned_df = calculated_df.copy()           
        
    cleaned_df['price'] = round(((cleaned_df['price']) / 1000000000),2)     #Arrondi et mise en forme                     
    cleaned_df.rename(columns={'price': 'Price [Md€]'}, inplace=True)       
    
    cleaned_df['amount'] = round((cleaned_df['amount']),2)                  #Arrondi et mise en forme                                                 
    cleaned_df.rename(columns={'amount': 'Amount [BTC]'}, inplace=True)
    
    cleaned_df['sigma'] = round((cleaned_df['sigma']),2)                    #Arrondi et mise en forme                                                 

    cleaned_df[f'{exchange_name}_vwap'] = round((cleaned_df[f'{exchange_name}_vwap']),2)     #Arrondi    
   
    cleaned_df = cleaned_df[~cleaned_df.apply(                                              
        lambda row: all(val == 0.0 or pd.isnull(val) for val in row), axis=1        #Filtrage des lignes où toutes les valeurs sont nulles 
    )] 
                 
    return cleaned_df

def compilate_data(cleaned_dict, vwmp_type):     #(4) ASSEMBLAGE DES RESULTATS DANS des dataframes de synthese          
    
    df_synthese_vwap = None    #Dataframe vide pour VWAP
    df_synthese_vwmp = None    #Dataframe vide pour VWMP
    
    for key, df in cleaned_dict.items():   #SYNTHESE VWAP

        exchange_name = (str(key)).split("df_")[1]         
                                
        if f'{exchange_name}_vwap' in df.columns:         
            
            vwap_column = df[f'{exchange_name}_vwap']        #Extrait la colonne 'wvap' de l'exchange
                        
            if df_synthese_vwap is None:                          #1ère itération :
               df_synthese_vwap = vwap_column.to_frame()          #--> copie de la colonne 'vwap' de l'exchange et intégration au dataframe de synthese

            else:
               df_synthese_vwap = pd.concat([df_synthese_vwap, vwap_column], axis=1)  #itération >= 2 : --> concaténe la colonne 'wvap' de l'exchange au dataframe de synthese

    #-- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - #symétrie

    for key, df in cleaned_dict.items():   #SYNTHESE VWMP

        exchange_name = (str(key)).split("df_")[1]         
                            
        if f'{exchange_name}_vwmp ({vwmp_type})' in df.columns:         
            
            vwmp_column = df[f'{exchange_name}_vwmp ({vwmp_type})']        #Extrait la colonne 'wvmp' de l'exchange
                        
            if df_synthese_vwmp is None:                          #1ère itération :
               df_synthese_vwmp = vwmp_column.to_frame()          #--> copie de la colonne 'vwmp' de l'exchange et intégration au dataframe de synthese
            else:
               df_synthese_vwmp = pd.concat([df_synthese_vwmp, vwmp_column], axis=1)  #itération >= 2 : --> concaténe la colonne 'wvmp' de l'exchange au dataframe de synthese
    
    compilated_dict = cleaned_dict.copy()       #Duplication du dictionnaire d'entrée

    compilated_dict['df_synthese_vwap'] = df_synthese_vwap         #Intégration du dataframe 'synthese_vwap' dans le nouveau dictionnaire
    compilated_dict['df_synthese_vwmp'] = df_synthese_vwmp         #Intégration du dataframe 'synthese_vwmp' dans le nouveau dictionnaire
        
    return compilated_dict

3) Visualisation des données

In [21]:
def visualisation(cleaned_dict, max_table_height, max_visible_rows):
    
    for key, df in reversed(list(cleaned_dict.items())):
        
        exchange_name = key.split("df_")[1]     #Nom de l'exchange
        vwmp_column_index = 9                   #Index colonne 'vwmp' (constante)
        
        candlestick = go.Figure(data=[go.Candlestick(x=df.index,
                                                     open=df['price_open'],
                                                     high=df['price_high'],
                                                     low=df['price_low'],
                                                     close=df['price_close'])])

        candlestick.update_layout(
            title=f'BTC-EUR [{exchange_name}]',
            yaxis_title='Price',
            height=300,  
            width=700,   
            margin=dict(l=20, r=10, t=50, b=20),  
            plot_bgcolor='#F5F5F5',  
        )
        
        candlestick.add_trace(go.Scatter(x=df.index, y=df[f'{exchange_name}_vwap'], mode='lines', name='vwap', line_color='black'))                             #Courbe VWAP
        candlestick.add_trace(go.Scatter(x=df.index, y=df.iloc[:, vwmp_column_index], mode='lines', name=df.columns[vwmp_column_index], line_color='grey'))     #Courbe VWMP 
        
        graph_html = candlestick.to_html(full_html=False, include_plotlyjs='cdn')       #Convertion du graphique en HTML

        selected_columns = [0, 5, 7, 9, 8]      #Définition des colonnes souhaitées pour afficher la table
        df_selected = df.iloc[:, selected_columns].head(max_visible_rows)       #Filtrage du df pour affichage
        table_html = df_selected.to_html(index=True, classes=f'scrollable-table-container-{exchange_name}', max_rows=len(df_selected))  #Conversion du dataframe en table HTML

        #Concaténation de la chaine HTML pour affichage Table/Graphique de chaque df
        html_output = f"""                                                                          
            <div style="display: flex; flex-direction: row;">                                                   
                <div style="width: 45%; max-height: {max_table_height}px; overflow-y: auto;">
                    <h2 style="position: sticky; top: 0;">{exchange_name} exchange</h2>
                    <div style="overflow-x: auto;">
                        {table_html}
                    </div>
                </div>
                <div style="width: 55%; overflow: hidden;">{graph_html}</div>
            </div>
        """
        
        display(HTML(html_output))      #Affichage de la sortie HTML (table + graphique)

4) Export des dataframes au format csv

In [22]:
def export_csv(compilated_dict, frequency, vwmp_type):      

    notebook_dir = os.getcwd()      #Chemin du notebook
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")      #Timestamp du lancement export
    export_folder = (notebook_dir) + '/../output/Export_' + (now) + '_freq-'+ (frequency) + '_vwmp-' + (vwmp_type) +'/'     #Création d'un dossier 'export' horodaté avec les paramètres sélectionnées à l'export
    os.mkdir(export_folder)     #Création du dossier 'Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X'
                                                                                                    
    export_sucess = False 

    try:
        for key, df in compilated_dict.items():     #Pour chaque dataframe :    

            filename = (f"{key}.csv")                   #ID du fichier csv = nom du dataframe
            file_path = export_folder + filename        #Chemin de l'enregistrement
            df.to_csv(file_path, index=True)            #enregistre le dataframe en fichier .csv

        export_success = True
        print(f"Export des fichiers csv avec succès :\n\n --> {export_folder}\n\nPROJET-PYTHON\n├── data\n│   ├── bfly.csv\n│   ├── bfnx.csv\n│   └── ...\n├── livrable\n│   └── livrable_Marius_Ayrault.ipynb\n├── output\n│   └──Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X [Exported here]<--\n│       ├── df_all.csv\n│       ├── df_bfnx.csv\n│       ├── ...\n│       └── df_synthese.csv\n└── readme.md\n")
        return export_sucess
    
    except:
        export_success = False
        print("Erreur lors de l'export")
        return export_sucess 

#### RESULTATS

1) Pour définir les paramètres de calcul et lancer l'aggrégation des données :

In [23]:
frequency = get_frequency()     #Choix de l'interval d'aggrégation
vwmp_type = get_vwmp_type()     #Choix du type de calcul du VWMP (lower or upper)

aggregated_dict, calculated_dict, cleaned_dict, compilated_dict = process_data(all_dataframes_dict, frequency, vwmp_type)   #Aggrégation des données et calcul

Calcul terminé !

Fréquence d'aggrégation : 30min
Calcul du VWMP : lower


2) Pour visualiser les résultats :

In [24]:
visualisation(cleaned_dict, max_table_height=300, max_visible_rows=1000)

,Price [Md€],Amount [BTC],all_vwap,all_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,2.00,3107.08,47920.53,47694.66,645.13
2021-02-24 00:30:00,1.26,1416.78,47800.29,48226.20,288.71
2021-02-24 01:00:00,1.52,2210.72,48928.49,49312.94,233.03
2021-02-24 01:30:00,1.95,2628.34,49637.09,49658.79,356.48
2021-02-24 02:00:00,2.01,2669.20,50210.20,50002.40,204.75
2021-02-24 02:30:00,1.42,1737.96,50296.35,50270.33,99.57
2021-02-24 03:00:00,1.18,1513.76,50375.68,50182.39,102.27
2021-02-24 03:30:00,1.27,1561.65,50647.81,50705.07,235.35
2021-02-24 04:00:00,1.60,2364.35,51093.51,51024.81,161.34


,Price [Md€],Amount [BTC],krkn_vwap,krkn_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.13,307.02,47627.94,47605.1,480.47
2021-02-24 00:30:00,0.09,177.18,47820.24,47598.7,310.88
2021-02-24 01:00:00,0.11,315.04,48861.94,48748.8,222.15
2021-02-24 01:30:00,0.13,260.49,49562.51,49521.6,363.03
2021-02-24 02:00:00,0.15,352.76,50189.33,50455.9,216.11
2021-02-24 02:30:00,0.07,131.70,50281.94,50200.0,102.59
2021-02-24 03:00:00,0.05,132.06,50379.80,50500.0,96.97
2021-02-24 03:30:00,0.08,165.77,50662.31,50823.8,237.17
2021-02-24 04:00:00,0.11,222.71,51094.75,51068.5,148.21


,Price [Md€],Amount [BTC],cbse_vwap,cbse_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.84,1333.18,48004.05,47329.95,635.65
2021-02-24 00:30:00,0.61,635.79,47822.24,47613.76,279.12
2021-02-24 01:00:00,0.69,854.33,48943.96,48940.71,237.31
2021-02-24 01:30:00,0.83,1045.92,49622.08,49853.12,357.64
2021-02-24 02:00:00,0.97,1140.98,50211.59,50472.43,207.62
2021-02-24 02:30:00,0.76,763.72,50304.63,50400.81,98.18
2021-02-24 03:00:00,0.66,686.83,50379.59,50347.82,95.31
2021-02-24 03:30:00,0.63,663.53,50620.86,50412.42,243.56
2021-02-24 04:00:00,0.79,1181.79,51100.97,51238.39,156.88


,Price [Md€],Amount [BTC],bnus_vwap,bnus_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.09,69.58,47941.85,48957.22,665.98
2021-02-24 00:30:00,0.06,58.28,47733.24,48104.12,293.64
2021-02-24 01:00:00,0.08,65.99,48892.37,49155.12,232.07
2021-02-24 01:30:00,0.08,58.07,49541.45,49498.93,325.68
2021-02-24 02:00:00,0.09,73.48,50156.21,50002.94,203.89
2021-02-24 02:30:00,0.06,39.26,50283.16,50207.44,98.83
2021-02-24 03:00:00,0.05,34.78,50345.60,50285.39,98.69
2021-02-24 03:30:00,0.05,44.62,50661.82,50444.24,227.89
2021-02-24 04:00:00,0.09,70.27,51074.62,51009.42,157.66


,Price [Md€],Amount [BTC],bfly_vwap,bfly_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.0,1.01,47976.36,47743.62,563.32
2021-02-24 00:30:00,0.0,0.95,48039.65,48128.38,281.62
2021-02-24 01:00:00,0.0,0.21,48876.09,48798.70,67.72
2021-02-24 01:30:00,0.0,0.83,49793.95,50161.64,304.41
2021-02-24 02:00:00,0.0,1.77,50196.68,49994.55,130.52
2021-02-24 03:30:00,0.0,0.25,50948.00,50948.00,0.00
2021-02-24 04:00:00,0.0,0.77,50892.81,51025.46,140.00
2021-02-24 06:00:00,0.0,0.13,49698.24,49699.15,181.00
2021-02-24 06:30:00,0.0,1.44,50004.36,50018.90,33.11


...

,Price [Md€],Amount [BTC],okcn_vwap,okcn_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.02,33.19,47943.65,48166.71,617.51
2021-02-24 00:30:00,0.01,18.14,47975.01,48199.35,266.06
2021-02-24 01:00:00,0.01,14.58,48886.50,48592.20,211.71
2021-02-24 01:30:00,0.01,21.81,49781.33,49775.26,326.86
2021-02-24 02:00:00,0.02,17.00,50149.92,50275.95,194.16
2021-02-24 02:30:00,0.01,18.84,50280.92,50057.64,100.81
2021-02-24 03:00:00,0.01,12.37,50348.22,50341.88,86.73
2021-02-24 03:30:00,0.01,4.99,50651.14,50527.52,232.58
2021-02-24 04:00:00,0.01,17.30,51148.92,51254.00,124.69


,Price [Md€],Amount [BTC],itbi_vwap,itbi_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.04,18.21,47792.65,48200.00,652.21
2021-02-24 00:30:00,0.04,9.09,47731.40,47806.75,262.89
2021-02-24 01:00:00,0.04,17.75,49053.23,49016.25,244.52
2021-02-24 01:30:00,0.04,42.61,49699.86,49700.00,340.01
2021-02-24 02:00:00,0.04,27.77,50259.69,50467.00,194.26
2021-02-24 02:30:00,0.04,11.45,50261.77,50333.25,96.35
2021-02-24 03:00:00,0.03,8.18,50335.97,50326.75,95.16
2021-02-24 03:30:00,0.03,11.53,50674.24,50940.00,237.39
2021-02-24 04:00:00,0.04,28.89,51100.20,51012.00,148.22


,Price [Md€],Amount [BTC],gmni_vwap,gmni_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.09,127.63,48088.06,47064.77,673.46
2021-02-24 00:30:00,0.05,56.05,47739.53,47948.71,240.56
2021-02-24 01:00:00,0.05,81.64,48920.44,49199.99,239.49
2021-02-24 01:30:00,0.07,131.28,49792.44,49953.63,374.07
2021-02-24 02:00:00,0.08,92.25,50243.57,50520.30,204.47
2021-02-24 02:30:00,0.05,49.97,50313.84,50323.07,97.61
2021-02-24 03:00:00,0.05,65.82,50419.05,50523.54,110.12
2021-02-24 03:30:00,0.04,44.30,50606.33,50792.01,242.24
2021-02-24 04:00:00,0.06,79.72,51091.57,50969.05,157.70


,Price [Md€],Amount [BTC],lmax_vwap,lmax_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.05,513.50,47923.72,48922.50,696.22
2021-02-24 00:30:00,0.03,181.54,47760.00,47615.50,232.67
2021-02-24 01:00:00,0.02,178.72,48832.62,48514.00,224.06
2021-02-24 01:30:00,0.02,189.34,49645.59,50020.00,341.10
2021-02-24 02:00:00,0.04,224.74,50178.73,50210.00,206.75
2021-02-24 02:30:00,0.03,193.44,50284.93,50377.50,96.75
2021-02-24 03:00:00,0.02,139.68,50370.16,50381.75,88.36
2021-02-24 03:30:00,0.03,172.01,50530.17,50462.50,250.21
2021-02-24 04:00:00,0.03,171.50,51099.15,51189.50,150.56


,Price [Md€],Amount [BTC],stmp_vwap,stmp_vwmp (lower),sigma
timestamp,,,,,
2021-02-24 00:00:00,0.06,225.27,47982.61,48702.61,673.30
2021-02-24 00:30:00,0.03,89.09,47847.30,47753.52,294.21
2021-02-24 01:00:00,0.07,348.21,48958.77,49300.00,231.22
2021-02-24 01:30:00,0.07,257.52,49573.94,49743.33,356.33
2021-02-24 02:00:00,0.08,269.30,50263.02,50324.86,210.07
2021-02-24 02:30:00,0.05,209.38,50309.97,50092.97,92.87
2021-02-24 03:00:00,0.05,229.46,50376.11,50405.62,87.78
2021-02-24 03:30:00,0.04,117.78,50660.69,50822.83,231.32
2021-02-24 04:00:00,0.07,263.05,51112.57,51313.90,167.65


3) Pour afficher le df de synthese du VWAP :

In [25]:
df_synthese_vwap = compilated_dict['df_synthese_vwap']
df_head = df_synthese_vwap.head(10)

def highlight_min(s):           #Fonction pour surligner la valeur minimale de chaque ligne
    is_min = s == s.min()
    return ['background-color: green' if v else '' for v in is_min]

styled_df = df_head.style.apply(highlight_min, axis=1)      #Applique le style conditionnel au dataframe

html_str = f"<h2>Synthèse des résultats [VWAP]</h2>"                                        #Conversion du dataframe stylisé en chaine HTML
html_str += styled_df.to_html(classes='scrollable-table', justify='left', escape=False)

display(HTML(html_str)) # Affichage du dataframe de synthese du [VWAP]

,stmp_vwap,lmax_vwap,gmni_vwap,itbi_vwap,okcn_vwap,bfnx_vwap,btrx_vwap,bfly_vwap,bnus_vwap,cbse_vwap,krkn_vwap,all_vwap
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,47982.610000,47923.720000,48088.060000,47792.650000,47943.650000,47793.410000,47864.800000,47976.360000,47941.850000,48004.050000,47627.940000,47920.530000
2021-02-24 00:30:00,47847.300000,47760.000000,47739.530000,47731.400000,47975.010000,47743.430000,47849.230000,48039.650000,47733.240000,47822.240000,47820.240000,47800.290000
2021-02-24 01:00:00,48958.770000,48832.620000,48920.440000,49053.230000,48886.500000,48980.560000,48911.970000,48876.090000,48892.370000,48943.960000,48861.940000,48928.490000
2021-02-24 01:30:00,49573.940000,49645.590000,49792.440000,49699.860000,49781.330000,49684.930000,49662.880000,49793.950000,49541.450000,49622.080000,49562.510000,49637.090000
2021-02-24 02:00:00,50263.020000,50178.730000,50243.570000,50259.690000,50149.920000,50206.020000,50253.130000,50196.680000,50156.210000,50211.590000,50189.330000,50210.200000
2021-02-24 02:30:00,50309.970000,50284.930000,50313.840000,50261.770000,50280.920000,50281.110000,50298.320000,nan,50283.160000,50304.630000,50281.940000,50296.350000
2021-02-24 03:00:00,50376.110000,50370.160000,50419.050000,50335.970000,50348.220000,50342.470000,50456.210000,nan,50345.600000,50379.590000,50379.800000,50375.680000
2021-02-24 03:30:00,50660.690000,50530.170000,50606.330000,50674.240000,50651.140000,50753.200000,50708.980000,50948.000000,50661.820000,50620.860000,50662.310000,50647.810000
2021-02-24 04:00:00,51112.570000,51099.150000,51091.570000,51100.200000,51148.920000,51046.870000,51108.930000,50892.810000,51074.620000,51100.970000,51094.750000,51093.510000


In [28]:
df_synthese_vwmp = compilated_dict['df_synthese_vwmp']
df_head = df_synthese_vwmp.head(10)

def highlight_min(s):           #Fonction pour surligner la valeur minimale de chaque ligne
    is_min = s == s.min()
    return ['background-color: green' if v else '' for v in is_min]

styled_df = df_head.style.apply(highlight_min, axis=1)      #Applique le style conditionnel au dataframe

html_str = f"<h2>Synthèse des résultats [VWMP] ({vwmp_type})</h2>"                          #Conversion du dataframe stylisé en chaine HTML
html_str += styled_df.to_html(classes='scrollable-table', justify='left', escape=False)

display(HTML(html_str))     #Affichage du dataframe de synthese du [VWAP]

,stmp_vwmp (lower),lmax_vwmp (lower),gmni_vwmp (lower),itbi_vwmp (lower),okcn_vwmp (lower),bfnx_vwmp (lower),btrx_vwmp (lower),bfly_vwmp (lower),bnus_vwmp (lower),cbse_vwmp (lower),krkn_vwmp (lower),all_vwmp (lower)
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48702.610000,48922.500000,47064.770000,48200.000000,48166.710000,47800.000000,47756.180000,47743.620000,48957.220000,47329.950000,47605.100000,47694.660000
2021-02-24 00:30:00,47753.520000,47615.500000,47948.710000,47806.750000,48199.350000,48420.000000,47534.860000,48128.380000,48104.120000,47613.760000,47598.700000,48226.200000
2021-02-24 01:00:00,49300.000000,48514.000000,49199.990000,49016.250000,48592.200000,48955.000000,49252.000000,48798.700000,49155.120000,48940.710000,48748.800000,49312.940000
2021-02-24 01:30:00,49743.330000,50020.000000,49953.630000,49700.000000,49775.260000,49900.000000,49670.600000,50161.640000,49498.930000,49853.120000,49521.600000,49658.790000
2021-02-24 02:00:00,50324.860000,50210.000000,50520.300000,50467.000000,50275.950000,50003.190000,50418.680000,49994.550000,50002.940000,50472.430000,50455.900000,50002.400000
2021-02-24 02:30:00,50092.970000,50377.500000,50323.070000,50333.250000,50057.640000,50151.000000,50179.600000,nan,50207.440000,50400.810000,50200.000000,50270.330000
2021-02-24 03:00:00,50405.620000,50381.750000,50523.540000,50326.750000,50341.880000,50281.000000,50488.860000,nan,50285.390000,50347.820000,50500.000000,50182.390000
2021-02-24 03:30:00,50822.830000,50462.500000,50792.010000,50940.000000,50527.520000,50792.000000,50777.000000,50948.000000,50444.240000,50412.420000,50823.800000,50705.070000
2021-02-24 04:00:00,51313.900000,51189.500000,50969.050000,51012.000000,51254.000000,51200.000000,51243.450000,51025.460000,51009.420000,51238.390000,51068.500000,51024.810000


4) Pour exporter les résultats au format csv :

In [27]:
export_success = export_csv(compilated_dict, frequency, vwmp_type)

Export des fichiers csv avec succès :

 --> /Users/mariusayrault/GitHub/Sorb-Data-Analytics/projet-python/livrable/../output/Export_2024-04-23_13-45-29_freq-30min_vwmp-lower/

PROJET-PYTHON
├── data
│   ├── bfly.csv
│   ├── bfnx.csv
│   └── ...
├── livrable
│   └── livrable_Marius_Ayrault.ipynb
├── output
│   └──Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X [Exported here]<--
│       ├── df_all.csv
│       ├── df_bfnx.csv
│       ├── ...
│       └── df_synthese.csv
└── readme.md

